Downloading Pre-Trained Model for BERT (Load Fine-Tuned BERT-large)

In [1]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

c:\Users\Resi Sujiwo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

End of Downloading Pre-Trained Model for BERT (Load Fine-Tuned BERT-large)

Importing Libraries

In [3]:
import torch
import pandas as pd 
from ast import literal_eval

# Question Translation Library
import translators as ts
import translators.server as tss

Using state Jakarta server backend.


Loading datasets

In [13]:
df = pd.read_csv('data/faq.csv', sep=";", converters={'paragraph': literal_eval})
# print(df.head())
paragraf = df['paragraphs']
judul = df['title']
nomor = df['No']
data = df['paragraphs']
data = " ".join(data)
data = data.replace("'", "")
print(paragraf)
print(data)

0     '' Hingga saat ini BINUS masih belum menyedia...
1     '' Pengajuan pemindahan program dapat dilakua...
2    "Proses pemindahan program dapat melalui beber...
3     '' Cara mengajukan pemindahan lokasi kampus m...
4     '' Syarat mengajukan pindah lokasi kuliah ole...
5     '' Mahasiswa dapat memilih lebih dari 1 progr...
6    "Calon Mahasiswa yang mengambil single degree ...
7    "Syarat Pendidikan terbagi menjadi 2 yaitu yan...
Name: paragraphs, dtype: object
  Hingga saat ini BINUS masih belum menyediakan jalur transfer pendaftaran.    Pengajuan pemindahan program dapat dilakuakn dengan dengan mengirimkan email permohonan pindah jurusan ke infobinus@binus.edu lalu akan diberikan formulir pindah program, setelah itu formulir yang sudah kalian isi bisa di scan dan kirimkan kembali.  "Proses pemindahan program dapat melalui beberapa proses berikut: 1.Proses pindah program akan dikenakan biaya administrasi sebesar Rp 3.000.000 (tiga juta rupiah) apabila: Permohonan pindah program 

In [18]:
def answer_question(question, datasets):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    question = tss.google(question, to_language='en')

    # for text in datasets:
    # print(text)
    text_string = tss.google(datasets, to_language='en')       
    # print(text_string)
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, text_string)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    # print("Start scoress", start_scores)
    # print("End scoress", end_scores)

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]
    # print("Answer Start Score", answer_start)

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    answer = tss.google(answer, to_language='id')

    for key, value in paragraf.items():
        if value.find(answer) != -1:
            print(f"The key of the answer is: {key}")

    print('Answer: "' + answer + '"')

In [21]:
question = "berapa biaya administrasi untuk melakukan pemindahan program?"

answer_question(question, data)

Query has 497 tokens.

Answer: "Rp 3, 000, 000"
